# Project 2 : :)
## Authors:

### Warnings

* Ikke endre uten at de andre vet det
* **Alltid** gå på Kernel -> Restart & Clear Output
* Ikke skriv utenfor cellene og sånn

## Implementation of set 1 - points in 2D

In [3]:
## IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import loader
import plotting
from spirals import get_data_spiral_2d

## GLOBAL CONSTS AND DEFINITIONS
sigma = lambda x : np.tanh(x)         # Activation function
eta = lambda x : 0.5*(1+np.tanh(x/2)) # Scalar function     
sech = lambda x : 1/np.cosh(x)
eta_ddx = lambda x :  0.25*sech(x/2)*sech(x/2) # Derivative of eta
sigma_ddx = lambda x: sech(x)*sech(x) # Derivative of sigma
h = 0.1                               # Step length          
#_____________________________________________________________

One = np.full((I,2),1)
#print(I)

In [4]:
# Algorithm
K = 4 # Amount of layers, arbitrary number
d = 3 # Ehh
tau = 0.07 # learning parameter [0.01,0.1]
Y0 = readData() # To be implemented
W, b = np.random.randn(K,d,d), np.random.randn(K,d,1)
converged = False
while not converged:
    for k in range(1,K):
        #Calculate Y[k] and save to memory
    getGradients()
    AdamAlg()
    converged = True #Only one iteration when debugging

IndentationError: expected an indented block (<ipython-input-4-f61faef558ce>, line 11)

In [10]:
from spirals import get_data_spiral_2d
import numpy as np
Y0, C = get_data_spiral_2d() # Default n=200
I = Y0.shape[1]
K=4 #forandrer på antall lag kun her
K = 4 #debug
mu = 1 #?
w = 1 #?
h = 0.1
d = 2

#optimert funksjon
def YK(K):
    #startverdier
    W, b = np.random.randn(K,d,d), np.random.randn(K,d,1)
    #tom Y-matrise, 3-dimensjonalt numpy-array
    Ym = np.zeros((K+1,d,I))
    Ym[0,:,:] = np.copy(Y0)
    k=0
    while k < K:
        Ym[k+1] = Ym[k] + h*sigma(W[k]@Ym[k] + b[k])
        k+=1
    return Ym

#print(YK(), "\n \n")
YKA= YK(3)
YKB = YK(4)
print(YKA)
print("ooooooooooooooooooooo")
print(YKB)





#-------------

#Funksjon for alle Y_k
def Y(k, W, b):
    #Y_all = []
    #Y_all.append(np.copy(Y0))
    assert k>=0
    if k==0:
        return np.copy(Y0)
    else:
        Y_km1 = Y(k-1, W, b)
        return Y_km1+h*sigma(W@Y_km1+b)
        #for i in range(k):
        #    Y_all.append(Y_all[i] + h*sigma(W[i]*Y_all[i] + b[i]))
        #return Y_all

        
#Lager siste Y_k = Y_K
#print(Y(K,W,b).shape)
YK_list = Y(K,W,b)
lastY = YK_list[K-1]
#print(lastY.shape)


Z = lambda w,mu : eta(np.add(np.transpose(lastY)*w,np.ones((200,2)))) # NB SPØR STUDASS
#print(":)))")
J = 1/2*np.linalg.norm(Z(w,mu)-C)**2 # Cost function
#print(J)

[[[ 0.47850916  0.13388465  0.16335856 ...  0.41121817 -0.48864634
   -0.25349414]
  [ 0.35776487  0.20820235  0.27104387 ...  0.55965684  0.09513711
    0.82880447]]

 [[ 0.56899306  0.2255956   0.25566963 ...  0.50502422 -0.39353559
   -0.15524096]
  [ 0.45471408  0.2925407   0.35685006 ...  0.6546106   0.06008692
    0.82552283]]

 [[ 0.66061004  0.31511996  0.34627333 ...  0.59925845 -0.30754122
   -0.0589365 ]
  [ 0.37891078  0.2070086   0.27184418 ...  0.57565733 -0.03453656
    0.73236371]]

 [[ 0.75869431  0.40034694  0.43533765 ...  0.69773177 -0.35730515
    0.01989941]
  [ 0.35043909  0.21819885  0.27751164 ...  0.54571791  0.03276198
    0.75924479]]]
ooooooooooooooooooooo
[[[ 0.47850916  0.13388465  0.16335856 ...  0.41121817 -0.48864634
   -0.25349414]
  [ 0.35776487  0.20820235  0.27104387 ...  0.55965684  0.09513711
    0.82880447]]

 [[ 0.3974719   0.06970366  0.09445076 ...  0.32494694 -0.51470209
   -0.33664938]
  [ 0.37933986  0.26308942  0.32377483 ...  0.59003441 

NameError: name 'W' is not defined

In [ ]:


## Adam descent algorithn
def AdamAlg():
    beta1 = 0.9
    beta2 = 0.999
    alpha = 0.01
    epsilon = 1e-8
    g = getGradients()
    if counter==1:
        lastM = 0
        lastV = 0
    m = beta1*lastM+(1-beta1)*g
    v = beta2*lastV+(1-beta2)*(g*g)
    mhat = m/(1-beta1**(counter)) # NB må altså 
    vhat = v/(1-beta2**(counter))
    U -= alpha*mhat/(np.sqrt(vhat)+epsilon)
    lastM = m
    lastV = v
    return U
#Riktig return??


#Finner P_K
def getGradients(K, W, b):
    Y_all = Y(K,W,b)
    lastY = Y_all[K-1,:,:]
    P = np.full(Y0.shape,0) # init matrix
    P[K] = w@np.transpose((Z(w,mu)-C)*eta_ddx(np.transpose(lastY)*w+mu*One))  #(7)
    #Endra til @
    dJdMU = eta_ddx(np.transpose(np.transpose(lastY)*w+mu*One))*(Z(w,mu)-C)   #(5)
    dJdw = lastY*((Z(w,mu)-C)*eta_ddx(np.transpose(lastY)*w+mu))              #(6)
    for k in range(K,0,-1):
        P[k-1] = P[k]+h*np.transpose(W[k-1,:,:])@(sigma_ddx(W[k-1,:,:]*Y_all[k-1,:,:]+b[k-1,:,:])*P[k])  #(8)  
        #Endra til @
        #Endra første W[k-a] ??
        # feil med 'Y(k-1,W,b)' ??? alle matriser i lag k-1??
    for k in range(0,K-1):
        dJdWk[k] = h(P[k+1]*sigma_ddx(W[k,:,:]*Y(k,W,b)+b[k,:,:]))@np.transpose(Y(k,W,b))
        dJdBk[k] = h(P[k+1]*sigma_ddx(W[k,:,:]*Y(k,W,b)+b[k,:,:]))@One
    return dJdWk, dJdBk, dJdw, dJdMU


# First for-loop in getGradients, is the range correct?
# Parameters for Adam alg?
# What is Y[k]? svart
# How to readData()? svart
# How to check if converged? And when? svart
# Hvordan få posisjon til farge? svart
# Må vi ha forskjellgie b_k? Hva menes med b_{k-1}?
# Hva gjør egentlig Adam Descent Alg.? (Hva er U?)
# Random startverdier for vekter og bias: på hvilket intervall??
# Hvor mange verdier for Bk og Wk må vi lage?
# Er Wk=2x2 og Bk=2x1?

In [ ]:
#### OLD BUT GOLD ;)


# Old adam algorithm
    g = [0 for i in range(K)]
    m = [0 for i in range(K)]
    v = [0 for i in range(K)]
    mhat = [0 for i in range(K)]
    vhat = [0 for i in range(K)]
    v[0] = 0
    m[0]
    for j in range(999):
        g[j] = uGrad(J(U[j]))
        m[j] = beta1*m[j-1]+(1-beta1)*g[j]
        v[j] = beta2*v[j-1]+(1-beta2)(g[j]*g[j])
        mhat[j] = m[j]/(1-beta1**(j+1))
        vhat[j] = v[j]/(1-beta2**(j+1))
        U[j+1] -= alpha*mhat[j]/(np.sqrt(vhat[j])+e)

In [ ]:
matrix1 = np.array([[1,2,3], [2,3,4]])
matrix2 = np.array([[1,4,8], [0,2,6], [1,6,2]])
print(matrix1+matrix2)